# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
#!pip install chromadb langchain langchain_community langchain_openai

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/19.3 MB ? eta -:--:--
   --- ------------------------------------ 1.6/19.3 MB 9.4 MB/s eta 0:00:02
   -------- ------------------------------- 3.9/19.3 MB 10.7 MB/s eta 0:00:02
   ------------------ --------------------- 8.9/19.3 MB 15.0 MB/s eta 0:00:01
   ------------------------- -------------- 12.3/19.3 MB 15.1 MB/s eta 0:00:01
   ----------------------------------- ---- 17.3/19.3 MB 17.0 MB/s eta 0:00:01
   ---------------------------------------- 19.3/19.3 MB 16.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 21.5 M

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyabsa 2.4.1.post1 requires protobuf<4.0.0, but you have protobuf 5.29.5 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.


In [2]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [ ]:
# If you're using colab, run this
#os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

In [4]:
llm = OpenAI(temperature=0)

In [5]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "state_of_the_union.txt")

In [16]:
loader = TextLoader(doc_path, encoding='utf-8')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [17]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [18]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [19]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [21]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1049, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 2321, which is longer than the specified 1000


## Create the Agent

In [22]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [23]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [24]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

C:\Users\rospi\AppData\Local\Temp\ipykernel_24336\1834837320.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [28]:
agent.invoke(
    "What did they say about inflation"
)



> Entering new AgentExecutor chain...
 Inflation is a complex economic concept, so it would be best to use a QA system that specializes in economic topics.
Action: State of Union QA System
Action Input: "What did they say about inflation?"
Observation:  They said that inflation is robbing families of their gains and that it is a top priority to get prices under control. They also mentioned that one way to fight inflation is to drive down wages, but they have a better plan to lower costs and increase the productive capacity of the economy. They also mentioned that their plan has support from economists, business leaders, and most Americans.
Thought: This is helpful information, but I wonder if there is a more specific QA system for economic topics.
Action: Ruff QA System
Action Input: "What did they say about inflation?"
Observation:  I don't know.
Thought: It seems like the Ruff QA System is not specialized in economic topics. I should stick with the State of Union QA System.
Final A

{'input': 'What did they say about inflation',
 'output': "According to the State of Union address, the government's plan to fight inflation includes driving down wages and increasing the productive capacity of the economy."}

In [29]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 You should always think about the differences between ruff and flake8 before deciding which one to use.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff implements more rules than Flake8 and does not support custom lint rules. It also has some minor differences in its rule set and does not include all the 'opinionated' rules from flake8-bugbear. Additionally, Ruff is capable of automatically fixing its own lint violations, while Flake8 supports plugins and allows for custom and third-party rules.
Thought: Now that I know the differences, I can make an informed decision.
Final Answer: It ultimately depends on your specific needs and preferences, but some potential reasons to choose ruff over flake8 could include its larger rule set and automatic fixing capabilities.

> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': 'It ultimately depends on your specific needs and preferences, but some potential reasons to choose ruff over flake8 could include its larger rule set and automatic fixing capabilities.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [30]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [31]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [32]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation:  Biden said that he nominated Ketanji Brown Jackson for the United States Supreme Court and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': " Biden said that he nominated Ketanji Brown Jackson for the United States Supreme Court and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence."}

In [33]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 You should always think about the differences between ruff and flake8 before deciding which one to use.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff implements more rules than Flake8 and does not support custom lint rules. It also has some minor differences in its rule set and does not include all the 'opinionated' rules from flake8-bugbear. Additionally, Ruff is capable of automatically fixing its own lint violations, while Flake8 supports plugins and allows for custom and third-party rules.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': " Ruff implements more rules than Flake8 and does not support custom lint rules. It also has some minor differences in its rule set and does not include all the 'opinionated' rules from flake8-bugbear. Additionally, Ruff is capable of automatically fixing its own lint violations, while Flake8 supports plugins and allows for custom and third-party rules."}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [34]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [35]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [37]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union? Did Biden talk about inflation?"
)



> Entering new AgentExecutor chain...
 I should use the Ruff QA System to answer questions about ruff.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff does not support source.organizeImports and source.fixAll code actions in Jupyter Notebooks.
Thought: I should use the State of Union QA System to answer questions about the most recent state of the union address.
Action: State of Union QA System
Action Input: Did the president mention that tool in the state of the union?
Observation: 
No, the president did not mention a specific tool in the state of the union.
Thought: I should use the State of Union QA System to answer questions about the most recent state of the union address.
Action: State of Union QA System
Action Input: Did Biden talk about inflation?
Observation:  Yes, Biden mentioned inflation in the context of discussing his plan to fight it. He also mentioned how it affects families and the economy.
Thought: I now

{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union? Did Biden talk about inflation?',
 'output': 'Yes, Biden did mention inflation in the state of the union address.'}